In [1]:
# get data lists
from nipype.interfaces.ants import ANTS
from nipype.interfaces import fsl
import nipype.interfaces.io as nio

#PD_LIST='/codes/PD_template_subjects.list';
current_group='PD'
SUB_LIST = '/codes/'+current_group+'_template_subjects.list'
with open(SUB_LIST, 'r') as f_sub:
    sub_list = f_sub.readlines()
sub_list = [x[0:-1] for x in sub_list]

# file import
ds_T1 = nio.DataGrabber(infields=['subject_id', 'subject_id'])
ds_T1.inputs.base_directory = '/data/'+current_group # database
ds_T1.inputs.template = '%s/anat/%s_desc-preproc_T1w.nii.gz' # from cwd
ds_T1.inputs.sort_filelist = True
ds_T1.inputs.subject_id = sub_list
res_T1 = ds_T1.run()
t1_list=res_T1.outputs.outfiles
# file import
ds_9c = nio.DataGrabber(infields=['subject_id', 'subject_id'])
ds_9c.inputs.base_directory = '/data/'+current_group # database
ds_9c.inputs.template = '%s/anat/%s_space-MNI152NLin2009cAsym_desc-preproc_T1w.nii.gz' # from cwd
ds_9c.inputs.sort_filelist = True
ds_9c.inputs.subject_id = sub_list
res_9c = ds_9c.run()
res_9c_list=res_9c.outputs.outfiles
# file import
ds_mask = nio.DataGrabber(infields=['subject_id', 'subject_id'])
ds_mask.inputs.base_directory = '/data/'+current_group # database
ds_mask.inputs.template = '%s/anat/%s_desc-brain_mask.nii.gz'# from cwd
ds_mask.inputs.sort_filelist = True
ds_mask.inputs.subject_id = sub_list
res_mask = ds_mask.run()
t1_mask_list=res_mask.outputs.outfiles

In [5]:
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage
import os,time

OUT_DIR='/output/sMRI/test/'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
atlas_9c='/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c.nii'
atlas_9c_mask='/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'
atlas_09_masked='/output/sMRI/t1_MNI2009c_masked.nii.gz'
mask = fsl.ApplyMask(
    in_file=atlas_9c,
    out_file=atlas_09_masked,
    mask_file=atlas_9c_mask)
mask.run()

n_sub = len(t1_list)

#strings
i_sub=0
str_sub = t1_list[i_sub]
t1_masked = '/output/sMRI/test/sub-0094_desc-preproc_T1w-masked.nii.gz'
print('t1_masked: ',t1_masked)
t1_deformed = '/output/sMRI/test/sub-0094_desc-preproc_T1w-masked_'+'space-MNI2009c.nii.gz'
print('t1_deformed: ',t1_deformed)
str_output_transform_prefix = '/output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c_'
print('str_output_transform_prefix: ',str_output_transform_prefix)
t1_warp   = str_output_transform_prefix+'Warp.nii.gz'
t1_affine = str_output_transform_prefix+'Affine.txt'
print('t1_warp: ', t1_warp)
print('t1_affine: ', t1_affine)
t1_deformed_jacobian = str_output_transform_prefix+'Jacobian.nii.gz'
print('t1_deformed_jacobian: ',t1_deformed_jacobian)
print('processing: ',str_sub, ':\n' )
#apply mask
#mask = fsl.ApplyMask(
#    in_file=t1_list[i_sub],
#    out_file=t1_masked,
#    mask_file=t1_mask_list[i_sub])
#mask.run()
#ants reg
t0=time.time()
ants = ANTS()
ants.inputs.dimension = 3
ants.inputs.num_threads = 4
ants.inputs.output_transform_prefix = str_output_transform_prefix
ants.inputs.metric = ['CC']
ants.inputs.fixed_image = [atlas_09_masked]
ants.inputs.moving_image = [t1_masked]
ants.inputs.metric_weight = [1.0]
ants.inputs.radius = [4]
ants.inputs.transformation_model = 'SyN'
ants.inputs.gradient_step_length = 0.25
ants.inputs.number_of_iterations = [50, 35, 20]
ants.inputs.use_histogram_matching = True
ants.inputs.mi_option = [32, 16000]
ants.inputs.regularization = 'Gauss'
ants.inputs.regularization_gradient_field_sigma = 3
ants.inputs.regularization_deformation_field_sigma = 0
ants.inputs.number_of_affine_iterations = [10000,10000,10000,10000,10000]
ants.cmdline
ants.run()
t1=time.time()
    
print('reg takes: ', t1-t0)

t1_masked:  /output/sMRI/test/sub-0094_desc-preproc_T1w-masked.nii.gz
t1_deformed:  /output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c.nii.gz
str_output_transform_prefix:  /output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c_
t1_warp:  /output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c_Warp.nii.gz
t1_affine:  /output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c_Affine.txt
t1_deformed_jacobian:  /output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c_Jacobian.nii.gz
processing:  /data/PD/sub-0094/anat/sub-0094_desc-preproc_T1w.nii.gz :

200818-22:23:43,210 nipype.interface INFO:
	 stdout 2020-08-18T22:23:43.210074: Run Reg 
200818-22:23:43,214 nipype.interface INFO:
	 stdout 2020-08-18T22:23:43.214399: values 1
200818-22:23:44,217 nipype.interface INFO:
	 stdout 2020-08-18T22:23:44.217819:  Fixed image file: /output/sMRI/t1_MNI2009c_masked.nii.gz
200818-22:23:45,445 nipype.interface INFO:
	 stdout 2020-08-18T22:23:45.4457

In [6]:
# apply deformation
at1 = ApplyTransforms()
at1.inputs.dimension = 3
at1.inputs.input_image = t1_masked
at1.inputs.reference_image = atlas_09_masked
at1.inputs.output_image = t1_deformed
at1.inputs.interpolation = 'BSpline'
at1.inputs.interpolation_parameters = (5,)
at1.inputs.default_value = 0
at1.inputs.transforms = [t1_warp, t1_affine]
at1.inputs.invert_transform_flags = [False, False]
at1.cmdline
at1.run()

200818-23:36:03,292 nipype.interface INFO:
	 stdout 2020-08-18T23:36:03.292287:Using double precision for computations.
200818-23:36:03,295 nipype.interface INFO:
	 stdout 2020-08-18T23:36:03.295125:Input scalar image: /output/sMRI/test/sub-0094_desc-preproc_T1w-masked.nii.gz
200818-23:36:03,754 nipype.interface INFO:
	 stdout 2020-08-18T23:36:03.754291:Reference image: /output/sMRI/t1_MNI2009c_masked.nii.gz
200818-23:36:06,70 nipype.interface INFO:
	 stdout 2020-08-18T23:36:06.070176:=============================================================================
200818-23:36:06,72 nipype.interface INFO:
	 stdout 2020-08-18T23:36:06.070176:The composite transform comprises the following transforms (in order): 
200818-23:36:06,73 nipype.interface INFO:
	 stdout 2020-08-18T23:36:06.070176:  1. /output/sMRI/test/sub-0094_desc-preproc_T1w-masked_space-MNI2009c_Affine.txt (type = MatrixOffsetTransformBase)
200818-23:36:06,75 nipype.interface INFO:
	 stdout 2020-08-18T23:36:06.070176:  2. /out

In [10]:
from nipype.interfaces import fsl
from nipype.interfaces.ants import ANTS, ApplyTransforms,CreateJacobianDeterminantImage
import os,time
t1_deformed_jacobian='/output/sMRI/test/Jacobian.nii.gz'
jacobian = CreateJacobianDeterminantImage()
jacobian.inputs.imageDimension = 3
jacobian.inputs.deformationField = t1_warp
jacobian.inputs.outputImage = t1_deformed_jacobian
jacobian.inputs.num_threads = 6
jacobian.cmdline
jacobian.run()

In [14]:
t1_deformed_jacobian_masked='/output/sMRI/test/Jacobian_masked.nii.gz'
mask = fsl.ApplyMask(
    in_file=t1_deformed_jacobian,
    out_file=t1_deformed_jacobian_masked,
    mask_file=atlas_9c_mask)
mask.run()

In [13]:
atlas_9c_mask

'/templateflow/atlas/mni_icbm152_nlin_asym_09c_nifti/mni_icbm152_nlin_asym_09c/mni_icbm152_t1_tal_nlin_asym_09c_mask.nii'

In [ ]:
# Jacobian of deformation field
t2=time.time()
jacobian = CreateJacobianDeterminantImage()
jacobian.inputs.imageDimension = 3
jacobian.inputs.deformationField = t1_warp
jacobian.inputs.outputImage = t1_deformed_jacobian
jacobian.inputs.num_threads = 6
jacobian.cmdline
jacobian.run()

#mask jacobian
mask = fsl.ApplyMask(
    in_file=t1_deformed_jacobian,
    out_file=str_output_transform_prefix+'Jacobian_masked.nii.gz',
    mask_file=t1_mask_list[i_sub])
mask.run()
print('Processing ', str_sub,' takes', time.time()-t0 )